<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EC%BC%88%EB%A6%AC%EA%B8%B0%EB%B0%98%EB%A7%A4%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


In [ ]:

def round_half_up_to_two(num):
    """숫자를 소수점 둘째 자리에서 반올림하는 함수"""
    try:
        if isinstance(num, (float, int)):
            num_100 = num * 100
            if num_100 - int(num_100) >= 0.5:
                return (int(num_100) + 1) / 100
            else:
                return int(num_100) / 100
        else:
            return num
    except:
        return num


In [ ]:

def get_data(ticker, start, end):
    """주가 데이터 다운로드"""


    df = yf.download(ticker, start=start, end=end)
    df = df.drop(columns=['Volume', 'Adj Close'])
    df.index = df.index.date

    # 호가 단위 0.01$ 적용
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].map(round_half_up_to_two)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change() * 100
    df['Return'] = df['Return'].map(round_half_up_to_two)

    return df


In [ ]:
def get_probability_from_updays(up_days):
    """상승 횟수에 따른 상승 확률 반환"""
    probability_table = {
        0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0,
        7: 0.0265,
        8: 0.1122,
        9: 0.3026,
        10: 0.4880,
        11: 0.7352,
        12: 0.8538,
        13: 0.9164,
        14: 0.9766,
        15: 0.9963,
        16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0
    }
    return probability_table.get(up_days, 0.0)



In [ ]:
def calculate_win_probability(df, current_idx):
    """승률 계산"""
    # 최근 20일 데이터
    lookback_data = df.iloc[max(0, current_idx-20):current_idx]
    lookback_size = len(lookback_data)

    # 상승한 날 수 계산
    up_days = (lookback_data['Return'].values > 0).sum()

    # 테이블에서 확률 가져오기
    win_prob = get_probability_from_updays(up_days)

    return win_prob

In [37]:
# 켈리 범위를 임시로 0.6~1.0 으로 조정 (계속 0일시 평단가가 고정됨)
def kelly_formula(win_prob, win_loss_ratio=1.0):
    """Kelly 공식을 통한 최적 비율 계산"""
    q = 1 - win_prob
    f = (win_prob * win_loss_ratio - q) / win_loss_ratio
    return max(0.6, f)  # 0.5~1 사이로 제한


In [29]:
def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period):
   funds = initial_funds
   one_buy_amount = initial_funds / buy_portion  # 6분할
   holdings = 0
   avg_price = 0
   sell_waiting_days = 0  # 매도 대기 일수


   # DataFrame 인덱스 초기화
   df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                        columns=['날짜', '시가', '고가', '종가', '등락율', '상승횟수', '승률', '켈리비율',
                               'LOC 매수', 'LOC 평단 매수','LOC 전일종가 매수', '수익 실현 매도', 'MOC 손절',
                               '보유 주식 수', '평균단가', '예수금', '총 평가액', '수익율(%)'])

   for i in range(start_idx, start_idx + simulation_period + 1):
       # 데이터 시리즈로 가져오기
       open_price = df['Open'].values[i]
       high_price = df['High'].values[i]
       close_price = df['Close'].values[i]
       return_val = df['Return'].values[i]

       # 전일 종가
       close_price_prev = df['Close'].values[i-1]

       # 기본 데이터 저장
       df_res.at[i, '날짜'] = df.index[i]
       df_res.at[i, '시가'] = float(open_price)
       df_res.at[i, '고가'] = float(high_price)
       df_res.at[i, '종가'] = float(close_price)
       df_res.at[i, '등락율'] = f"{round_half_up_to_two(float(return_val))}%"

       # 상승횟수, 승률, 켈리비율 계산
       lookback_data = df.iloc[max(0, i-20):i] # 오늘 기준 지난 20일간 데이터
       up_days = (lookback_data['Return'].values > 0).sum() # 상승횟수 계산
       win_prob = get_probability_from_updays(up_days) # 표에 의한 승률
       kelly_ratio = kelly_formula(win_prob) # 켈리 공식에 의한 비율

       df_res.at[i, '상승횟수'] = up_days
       df_res.at[i, '승률'] = f"{round_half_up_to_two(win_prob * 100)}%"
       df_res.at[i, '켈리비율'] = f"{round_half_up_to_two(kelly_ratio * 100)}%"

       price = float(close_price) # 종가
       prev_price = float(close_price_prev) # 전날 종가
       optimal_amount = one_buy_amount * kelly_ratio # 1회차 구매액

       # 매수/매도 칼럼 초기화
       df_res.at[i, 'LOC 매수'] = 0
       df_res.at[i, 'LOC 평단 매수'] = 0
       df_res.at[i, 'LOC 전일종가 매수'] = 0
       df_res.at[i, '수익 실현 매도'] = 0
       df_res.at[i, 'MOC 손절'] = 0

       # 매수 로직
       if holdings == 0:
           # 첫 매수는 현재가 +15%에 LOC 매수
           loc_price = prev_price*(1.15)  # LOC 매수 희망가 (임시로 전날 종가의 15%로 LOC 매수 하는걸로 설정)
           if price <= loc_price:  # 종가가 LOC 매수 희망가보다 낮으면 종가에 매수
               qty = int(optimal_amount / price)
               if funds >= qty * price:
                   holdings += qty
                   funds -= qty * price
                   avg_price = ((avg_price * (holdings - qty)) + (price * qty)) / holdings if holdings > 0 else price
                   df_res.at[i, 'LOC 매수'] = qty
       else:
           # 분할 매수
           half_amount = optimal_amount / 2

           # 전일 종가에 LOC 매수
           if price < prev_price:  # 전일종가 보다 낮으면 매수
               qty1 = int(half_amount / price)
               if funds >= qty1 * price:
                   holdings += qty1
                   funds -= qty1 * price
                   avg_price = ((avg_price * (holdings - qty1)) + (price * qty1)) / holdings if holdings > 0 else price
                   df_res.at[i, 'LOC 전일종가 매수'] = qty1

           # 평단가에 LOC 매수
           if price <= avg_price:  # 종가가 평단가보다 낮으면 매수
               qty2 = int(half_amount / price)
               if funds >= qty2 * price:
                   holdings += qty2
                   funds -= qty2 * price
                   avg_price = ((avg_price * (holdings - qty2)) + (price * qty2)) / holdings if holdings > 0 else price
                   df_res.at[i, 'LOC 평단 매수'] = qty2

       # 매도 로직
       if holdings > 0:
           sell_waiting_days += 1

           # 수익이 수수료의 2배 이상일 때 전량 매도
           loc_sell_price = avg_price * (1.0018)  # LOC 매도 희망가 ( 한투 수수료 0.09 * 2 = 0.18 %)
           if price >= loc_sell_price:  # 종가가 LOC 매도가보다 높으면 LOC 매도가에 매도
               funds += holdings * price # 종가에 매도
               df_res.at[i, '수익 실현 매도'] = holdings
               holdings = 0
               avg_price = 0
               sell_waiting_days = 0

           # 30일 경과시 MOC 손절
           elif sell_waiting_days >= 30:
               funds += holdings * price  # 시장가(종가)에 매도
               df_res.at[i, 'MOC 손절'] = holdings
               holdings = 0
               avg_price = 0
               sell_waiting_days = 0

       # 포트폴리오 상태 저장
       df_res.at[i, '보유 주식 수'] = holdings
       df_res.at[i, '평균단가'] = round_half_up_to_two(avg_price)
       df_res.at[i, '예수금'] = round_half_up_to_two(funds)
       df_res.at[i, '총 평가액'] = round_half_up_to_two(funds + (price * holdings))
       df_res.at[i, '수익율(%)'] = round_half_up_to_two((funds + (price * holdings)) / initial_funds - 1) * 100

   final_value = funds + (holdings * float(df['Close'].values[start_idx + simulation_period]))
   return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value

In [39]:

# 메인 실행 코드
if __name__ == "__main__":
    start_date = '2020-01-01'
    end_date = '2022-01-31'
    initial_funds = 40000
    buy_portion = 6  # 6분할

    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
    next_day = end_date_dt + timedelta(days=1)
    end_day_next = next_day.strftime('%Y-%m-%d')

    # 시작일 30일 전의 날짜
    start_date_dt = datetime.strptime(start_date,'%Y-%m-%d')
    start_date_before_30 = start_date_dt - timedelta(days=30)
    start_date_before_30 = start_date_before_30.strftime('%Y-%m-%d')

    # 30일 전 데이터부터 입력
    df = get_data(ticker='TQQQ', start=start_date_before_30, end=end_day_next)
    # 늘어난 날짜 수 - 원래 날짜 수
    df_length =len(df) - len(get_data(ticker='TQQQ', start=start_date, end=end_day_next))
    df_res = pd.DataFrame(columns=['날짜', '시가', '고가', '종가', '등락율', 'LOC 매수', 'LOC 평단 매수', 'LOC 전일종가 매수'
                                  '수익 실현 매도', 'MOC 손절', '보유 주식 수', '평균단가', '예수금',
                                  '총 평가액', '수익율(%)'])

    return_rate, df_res, final_value = infinite_buy_simulation(df, df_res, initial_funds, buy_portion, df_length, len(df)-1-df_length)

    # 중앙 정렬하여 출력
    df_res_style = df_res.style.set_properties(**{'text-align': 'center'})
    display(df_res_style)

    print('\n' + '='*80)
    print(f"{start_date} ~ {end_date} 동안의 자산 변동 결과")
    print('='*80)
    print(f"최초 보유 금액: ${initial_funds:,.2f}")
    print(f"최종 보유 금액: ${final_value:,.2f}")
    print(f"원금 변화율: {return_rate}%")
    print('='*80)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-29-da889d130be3>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df_res.at[i, '시가'] = float(open_price)
<ipython-input-29-da889d130be3>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df_res.at[i, '고가'] = float(high_price)
<ipython-input-29-da889d130be3>:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df_res.


2020-01-01 ~ 2022-01-31 동안의 자산 변동 결과
최초 보유 금액: $40,000.00
최종 보유 금액: $39,457.21
원금 변화율: -1.35%
